# Building Database for Crime Reports
The goal of thisproject is to create a database named `crimes_db` with a table – `boston_crimes` – with appropriate datatypes for storing the data from the boston.csv file. 
We will be creating the table inside a schema named `crimes`. 

We will create two user groups:
- `readonly`: Users in this group will have permission to read data only.
- `readwrite`: Users in this group will have permissions to read and alter data but not to delete tables.

Finally, create one user for each of these groups. 

The following diagram illustrates a high-level overview of what we want to achieve:

![image](https://dq-content.s3.amazonaws.com/250/goal.png)

## Create Database and Schema

Create a database named `crime_db` and a schema named `crimes` for storing the tables for containing the crime data.

The database `crime_db` does not exist yet so we connect to `dq`.

In [1]:
import psycopg2
conn = psycopg2.connect(dbname='dq', user='dq')
conn.autocommit = True
cur = conn.cursor()

# delete user, group, and database if exist before we create new.
cur.execute('DROP USER IF EXISTS data_analyst, data_scientist')
cur.execute('DROP DATABASE IF EXISTS crime_db')
cur.execute('DROP GROUP IF EXISTS readonly, readwrite')

cur.execute('CREATE DATABASE crime_db')
conn.autocommit = False

In [2]:
conn = psycopg2.connect(dbname='crime_db', user='dq')
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## Exploring the CSV Data Set

Obtain the header row and assign it to a variable named `col_headers` and obtain the first data row and assign it to a variable named `first_row`.

In [3]:
import csv
#read_file = csv.reader('boston.csv')


with open('boston.csv', 'r') as file:
    reader = csv.reader(file)
    file_lst = list(reader)
    col_headers = file_lst[0]
    first_row = file_lst[1]
    
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


## Creating a function for analyzing column values
Create a function `get_col_set` that given a CSV file name and a column index computes the set of all distinct values in that column.

Use the function on each column to evaluate which columns have a lot of different values. Columns with a limited set of possible values are good candidates for enumerated datatypes.

In [4]:
def get_col_set(csv_filename, col_index):
    import csv
    with open(csv_filename, 'r') as file:
        next(file)
        reader = csv.reader(file)
        col_set = set()
        for row in reader:
            col_set.add(row[col_index])
    return col_set

## Analyzing the maximum length of the every columns
We use the `get_col_set` function to compute the maximum length to decide an appropriate length for that field.
From the result which displays unique values, maximum character length of and example values, we can justify which datatype suitable for every columns.

In [5]:
print(('Columns' +'\t'+ 'Counts' +'\t'+ 'Max Chars' +'\t'+ 'Max Output').expandtabs(17))
print("="*70)
for i in range(7):
    values = str(len(get_col_set('boston.csv', i)))
    max_len = 0
    the_max = ''
    for s in get_col_set('boston.csv', i):
        length = len(s)
        max_len = max(max_len, length)
        if max_len == length:
            the_max = s

    print((col_headers[i] +'\t'+ values +'\t'+ str(max_len) + '\t'+ the_max).expandtabs(17))
    

Columns          Counts           Max Chars        Max Output
incident_number  298329           6                212703
offense_code     219              4                2003
description      239              58               RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)
date             1177             10               2017-12-25
day_of_the_week  7                9                Wednesday
lat              18177            11               42.35326248
long             18177            12               -71.08544157


## Creating the table
We have create an enumerated datatype named weekday for the `day_of_the_week` since there there only 7 possible values.

For the `incident_number` we have decided to user the type `INTEGER` and set it as the primary key. The same datatype was also used to represent the `offense_code`.

Since the `description` has at most `58` character we decided to use the datatype `VARCHAR(70)` for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The `date` was represented as the `DATE` datatype. Finally, for the latitude 'lat` and 'long` longitude we used `DECIMAL` datatypes.

In [6]:
cur.execute('''CREATE TYPE days_enum AS ENUM (
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
    );''')

cur.execute("""
CREATE TABLE crimes.boston_crimes (
    incident_number INTEGER PRIMARY KEY,
    offense_code INTEGER,
    description VARCHAR(70),
    date DATE NOT NULL,
    day_of_the_week DAYS_ENUM,
    lat DECIMAL(10, 8),
    long DECIMAL(10, 8));
    """)

conn.commit()

## Load the data into the table
We used the copy_expert to load the data as it is very fast and very succinct to use.

In [7]:
with open("boston.csv", "r") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", f)
    
# print the number of rows and a sample to ensure that they were loaded
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))
print(cur.fetchone())

298329
None


## Revoke public privileges
We revoke all privileges of the public `public` group on the `public` schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the `public` schema.

We also need to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [8]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only group
We create a `readonly` group with `NOLOGIN` because it is a group and not a user. We grant the group the ability to connect to the `crime_db` and the ability to use the `crimes` schema.

Then we deal wit tables privileges by granting `SELECT`. We also add an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the `readonly` group on new table that are created on the `crimes` schema. As we mentioned, by default not privileges are given. However we change is so that by default any user in the `readonly` group can issue select commands.

In [9]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("REVOKE ALL ON ALL TABLES IN SCHEMA crimes FROM readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")


## Creating the read-write group
We create a `readwrite` group with `NOLOGIN` because it is a group and not a user. We grant the group the ability to connect to the `crime_db` and the ability to use the `crimes` schema.

Then we deal with tables privileges by granting `SELECT`, `INSERT`, `UPDATE` and `DELETE`. As before we change the default privileges so that user in the `readwrite` group have these privileges if we ever create a new table on the `crimes` schema.

In [10]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")  
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("REVOKE ALL ON ALL TABLES IN SCHEMA crimes FROM readwrite;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")


## Creating one user for each group
We create a user named `data_analyst` with password `secret1` in the `readonly` group.

We create a user named `data_scientist` with password `secret2` in the `readwrite` group.

We also alter both users permissions with `NOSUPERUSER` for better precaution.

In [11]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")
cur.execute("ALTER USER readonly WITH NOSUPERUSER;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_analyst;")
cur.execute("ALTER USER readwrite WITH NOSUPERUSER;")

In [12]:
# close the old connection to test with a brand new connection
conn.close()

## Test the database setup
Test the database setup using SQL queries on the `pg_roles` table and `information_schema.table_privileges`.

In the `pg_roles` table we will check database related privileges and for that we will look at the following columns:

- `rolname`: The name of the user / group that the privilege refers to.
- `rolsuper`: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
- `rolcreaterole`: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
- `rolcreatedb`: Whether user / group can create databases. It should be False on every user / group that we have created.
- `rolcanlogin`: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the `information_schema.table_privileges` we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [13]:
conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()

# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)


('data_analyst', False, False, False, True)
('readonly', False, False, False, False)
('data_scientist', False, False, False, True)
('readwrite', False, False, False, False)


In [14]:
# check privileges
cur.execute("""
    SELECT grantor, grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
print(cur.fetchall())

conn.close()

[('dq', 'readonly', 'SELECT'), ('dq', 'readwrite', 'INSERT'), ('dq', 'readwrite', 'SELECT'), ('dq', 'readwrite', 'UPDATE'), ('dq', 'readwrite', 'DELETE')]
